In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
import matplotlib.pyplot as plt

data_home = "/mnt/g/My Drive/GTC"

In [2]:
access = xr.open_dataset(f"{data_home}/ACCESS_data/input_data.nc")
ecco = xr.open_dataset(f"{data_home}/solodoch_data_minimal/60S.nc")

access = access[["sea_level", "surface_pot_temp", "surface_salt", "pbot_t"]]
access = access.drop_vars(["yt_ocean", "yu_ocean", "st_ocean"])
var_names = {"sea_level": "SSH",
             "pbot_t": "OBP",
             "surface_salt": "SSS",
             "surface_pot_temp": "SST",
             "xt_ocean": "longitude"}
access = access.rename(var_names)
latitude_value = ecco["latitude"].values[0]
latitude = xr.DataArray([latitude_value], dims="latitude")
access = access.assign_coords(latitude=latitude)

def find_nearest(array, value):
    array = np.asarray(array)
    ix = (np.abs(array - value)).argmin()
    return array[ix]

closest_lon = np.array([find_nearest(access.longitude, lon) for lon in ecco.longitude.to_numpy()])
access = access.sel(longitude=closest_lon, method="nearest")
access = access.groupby("longitude").mean()
access = access.interp(longitude=ecco.longitude, method="linear")
times = [t for t in access.time.values if t in ecco.time.values]
min_t = min(times); max_t = max(times)
access = access.sel(time=slice(min_t, max_t))
ecco = ecco.sel(time=slice(min_t, max_t))

In [3]:
for var in ["SSH", "SST", "SSS", "OBP"]:

    var_ecco = ecco.transpose("latitude", "time", "longitude")[var].to_numpy().squeeze()
    var_access = access.transpose("latitude", "time", "longitude")[var].to_numpy().squeeze()

    diff = var_ecco - var_access
    e_std = str(round(var_ecco[~np.isnan(var_ecco)].std(), 3))
    a_std = str(round(var_access[~np.isnan(var_access)].std(), 3))

    fig, ax = plt.subplots(figsize=(7, 7))
    sns.heatmap(diff.T, ax=ax, cmap="PiYG")
    ax.set_xticks([], []); ax.set_yticks([], [])
    ax.set_xlabel("time"); ax.set_ylabel("longitude")
    ax.set_title(f"{var}: Difference between\n ACCESS ($\\sigma$={a_std}) and ECCO ($\\sigma$={e_std})")
    plt.tight_layout()
    plt.savefig(f"graphs/{var}_access_vs_ecco.png", dpi=400)
    plt.close()